언어 모델 (Language Model) 은 텍스트 데이터의 통계 정보를 기반으로 자연어를 이해하고 생성하는 기능을 제공합니다.  
### 핵심 원리
언어모델의 기본 아이디어는 다음에 올 토큰 을 예측하는 것입니다. 예를 들어 '오늘 날씨가 정말 ____' 이라는 문장이 주어지면, 모델은 '좋다', '덥다' 같은 단어가 올 확률을 계산합니다. 이 단순한 원리가 충분히 큰 규모로 학습되면 놀라운 기능을 제공하게 됩니다. 

### 발전과정
#### 통계적 언어모델 (n-gram)
초기에는 이전 n개 단어만 보고 다음 단어를 예측했습니다. 단순하지만 긴 문맥을 이해하지 못하는 한계가 있었습니다. 
#### 신경망 기반 언어모델 (RNN/LSTM/GRU 등)
단어를 벡터로 표현하고  순환신경망을 통해 확률을 계산하는 방식입니다.  순차적으로 정보를 처리하며, 더 긴 문맥을 기억할 수 있게 되었지만, 긴 문장 학습이 어려웠고 병렬 처리가 어려워서 학습 속도가 느리다는 단점이 존재합니다.
#### Transformer (2017)
* "Attention is All You need" 라는 논문에서 핵심 아이디어가 도출되었습니다. Attention 매커니즘을 도입해 문장 내 모든 단어간의 관계를 병렬로 처리하게 되었습니다. 긴 문맥 이해가 가능하게 되었고 학습 속도가 급상승하게 되었습니다. 성능의 비약적 향상으로 LLM 의 기반 아키텍처가 됩니다. 

이후에도 언어 모델은 계속 발전하는 모습을 보여주고 있습니다. 

### Token
언어 모델의 기반 단위는 토큰입니다. 토큰은 모델에 따라 문자, 단어, 또는 단어의 일부가 될 수 있으며, GPT-4 인 경우는 토큰 하나의 평균 길이는 단어의 3/4 정도라고 말하고 있습니다. 

In [7]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
text="hello, my name is windfree."
tokens = encoding.encode(text)
print(tokens)

[15339, 11, 856, 836, 374, 10160, 10816, 13]


In [11]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"Token ID: {token_id}, Token Text:{token_text}")    

Token ID: 15339, Token Text:hello
Token ID: 11, Token Text:,
Token ID: 856, Token Text: my
Token ID: 836, Token Text: name
Token ID: 374, Token Text: is
Token ID: 10160, Token Text: wind
Token ID: 10816, Token Text:free
Token ID: 13, Token Text:.


토큰을 확인해 보았을 때 앞에 공백이 포함되는 경우를 확인할 수 있습니다. 이것은 위치 정보까지 토큰에 포함하기 위해서 입니다.  **토큰화** 는 원문을 모델이 정한 길이로 나누는 과정을 말하는 것이며 모델이 다룰 수 있는 토큰의 집합을 모델의 **어휘** 라고 부릅니다. 소수의 토큰을 사용해 많은 단어를 만들 수 있으며 GPT-4 의 어휘 크기는 100,256 개로 알려져 있습니다. 

## LLM ->  STATELESS 
### Memory 라는 환상
아래의 코드를 수행해보면 재미있는 현상을 발견할 수 있습니다. 


In [20]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("No API key was found")
else:
    print("API key found.")


API key found.


In [21]:
from openai import OpenAI
openai_client = OpenAI()
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, My name is windfree."}]
response = openai_client.chat.completions.create(
    model="gpt-4",
    messages=messages,)
print(response.choices[0].message.content)

Hello, Windfree! How can I assist you today?


In [22]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is my name?"}]
response = openai_client.chat.completions.create(
    model="gpt-4",
    messages=messages,)
print(response.choices[0].message.content)

I'm sorry, but as an AI I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.


첫번째 호출에서 내 이름을 말해준 후에 두번째 호출에서 내 이름을 물어보았을 때 LLM 은 내 이름을 모른다는 답을 하고 있습니다. 이유가 뭘까요? LLM 에 대한 모든 호출은 완전히 Stateless 한 상태입니다. 매번 완전히 새로운 호출인 셈이죠. LLM 이 "기억" 을 가진 것처럼 만드는 것은 AI 개발자의 몫입니다. 

In [24]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, My name is windfree."},
    {"role": "assistant", "content": "Hello, Windfree! How can I assist you today?"},
    {"role": "user", "content": "What is my name?"}]


In [25]:
response = openai_client.chat.completions.create(
    model="gpt-4",messages=messages)
print(response.choices[0].message.content)

Your name is Windfree.


당연한 얘기일 수 있지만, 다시 한번 정리해보면:
 * LLM에 대한 모든 호출은 무상태(stateless)다
 * 매번 지금까지의 전체 대화를 입력 프롬프트에 담아 전달한다.
 * 이게 LLM이 기억을 가진 것 같은 착각을 만든다 — 대화 맥락을 유지하는 것처럼 보이게 하지만 이건 트릭이다.
 * 매번 전체 대화를 제공한 결과일 뿐 LLM은 그저 시퀀스에서 다음에 올 가장 가능성 높은 토큰을 예측할 뿐이다. 
 * 시퀀스에 "내 이름은 windfree야"가 있고 나중에 "내 이름이 뭐지?"라고 물으면... windfree라고 예측하는 것!

많은 제품들이 정확히 이 트릭을 사용합니다. 메시지를 보낼 때마다 전체 대화가 함께 전달되는 겁니다. "그러면 매번 이전 대화 전체에 대해 추가 비용을 내야 하는 건가요?" 네. 당연히 그렇습니다. 그리고 그게 우리가 원하는 것이기도 합니다. 우리는 LLM이 전체 대화를 되돌아보며 다음 토큰을 예측하길 기대하고 있는 상태이며 그에 대한 사용료를 내야 하는 것입니다. 

실제로 LLM API를 다뤄보셨으니 체감하시겠지만, 매 요청마다 이전 대화 내역을 messages 배열에 다시 담아 보내는 구조가 바로 이 무상태성 때문입니다. 흔히 사용하는 "기억" 구현 기법들은 아래와 같습니다. 
 * 컨텍스트 주입: 이전 대화를 messages에 누적
 * 요약/압축: 긴 대화는 요약해서 system prompt에 삽입
 * RAG: 외부 저장소에서 관련 정보 검색 후 주입
 * 메모리 DB: 사용자별 중요 정보를 별도 저장 후 필요시 주입

 API 요금 구조를 보면 input token과 output token을 따로 과금하는데, 대화가 길어질수록 input token이 누적되어 비용이 기하급수적으로 늘어납니다. 그래서 실무에서는 대화 요약, sliding window, 오래된 메시지 삭제 같은 전략을 쓰게 됩니다.

동일한 작업을 로컬에 설치한 LLM 모델로 수행하는 코드를 만들어 보겠습니다. 로컬에 Ollama 를 설치해서 로컬 모델로 테스트해볼 수도 있습니다. 

In [29]:
ollama_url = "http://localhost:11434/v1"    
ollama_client = OpenAI(api_key="ollama", base_url=ollama_url)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, My name is windfree."},
    {"role": "assistant", "content": "Hello, Windfree! How can I assist you today?"},
    {"role": "user", "content": "What is my name?"}]
response = ollama_client.chat.completions.create(
    model="gpt-oss",messages=messages)
print(response.choices[0].message.content)

Your name is **windfree**.
